In [ ]:
function dlY = modelGenerator(dlX,parameters) dlY =
fullyconnect(dlX,parameters.FC.Weights,parameters.FC.Bias,'Dataformat','SSCB');

dlY = reshape(dlY,[1024 4 4 size(dlY,2)]);
dlY = permute(dlY,[3 2 1 4]); dlY = relu(dlY);

dlY = dltranspconv(dlY,parameters.Conv1.Weights,parameters.Conv1.Bias,'Stride'
,2 ,'Cropping','same','DataFormat','SSCB'); dlY = relu(dlY);

dlY = dltranspconv(dlY,parameters.Conv2.Weights,parameters.Conv2.Bias,'Stride'
,2 ,'Cropping','same','DataFormat','SSCB'); dlY = relu(dlY);

dlY = dltranspconv(dlY,parameters.Conv3.Weights,parameters.Conv3.Bias,'Stride'
,2 ,'Cropping','same','DataFormat','SSCB'); dlY = relu(dlY);

dlY = dltranspconv(dlY,parameters.Conv4.Weights,parameters.Conv4.Bias,'Stride'
,2 ,'Cropping','same','DataFormat','SSCB'); dlY = relu(dlY);

dlY = dltranspconv(dlY,parameters.Conv5.Weights,parameters.Conv5.Bias,'Stride'
,2 ,'Cropping','same','DataFormat','SSCB'); dlY = tanh(dlY);
end


function dlY = modelDiscriminator(dlX,parameters)

dlY = dlconv(dlX,parameters.Conv1.Weights,parameters.Conv1.Bias,'Stride' ,2
,'Padding','same');
dlY = leakyrelu(dlY,0.2);

dlY = dlconv(dlY,parameters.Conv2.Weights,parameters.Conv2.Bias,'Stride' ,2
,'Padding','same');
dlY = leakyrelu(dlY,0.2);

dlY = dlconv(dlY,parameters.Conv3.Weights,parameters.Conv3.Bias,'Stride' ,2
,'Padding','same');
dlY = leakyrelu(dlY,0.2);

dlY = dlconv(dlY,parameters.Conv4.Weights,parameters.Conv4.Bias,'Stride' ,2
,'Padding','same');
dlY = leakyrelu(dlY,0.2);

dlY = dlconv(dlY,parameters.Conv5.Weights,parameters.Conv5.Bias,'Stride' ,2
,'Padding','same');
dlY = leakyrelu(dlY,0.2);

dlY = stripdims(dlY);
dlY = permute(dlY,[3 2 1 4]);
dlY = reshape(dlY,4*4*64*16,numel(dlY)/(4*4*64*16));

weights = parameters.FC.Weights; bias = parameters.FC.Bias;
dlY = fullyconnect(dlY,weights,bias,'Dataformat','CB'); end
function gradientsDiscriminator = modelDiscriminatorGradients(discriminatorParameters , generatorParameters, dlX, dlZ)

% Calculate the predictions for real data with the discriminator network. dlYPred = modelDiscriminator(dlX,discriminatorParameters);

% Calculate the predictions for generated data with the discriminator network. dlXGenerated	= modelGenerator(dlZ,generatorParameters);
dlYPredGenerated = modelDiscriminator(dlarray(dlXGenerated,'SSCB'),discriminatorParameters);

% Calculate the GAN loss
lossDiscriminator = ganDiscriminatorLoss(dlYPred,dlYPredGenerated);

% For each network, calculate the gradients with respect to the loss. gradientsDiscriminator = dlgradient(lossDiscriminator,discriminatorParameters);

end

function gradientsGenerator = modelGeneratorGradients(discriminatorParameters, generatorParameters , dlZ)

% Calculate the predictions for generated data with the discriminator network. dlXGenerated = modelGenerator(dlZ,generatorParameters);

dlYPredGenerated = modelDiscriminator(dlarray(dlXGenerated,'SSCB'),discriminatorParameters);

% Calculate the GAN loss
lossGenerator = ganGeneratorLoss(dlYPredGenerated);

% For each network, calculate the gradients with respect to the loss. gradientsGenerator = dlgradient(lossGenerator, generatorParameters);

end
function lossDiscriminator = ganDiscriminatorLoss(dlYPred,dlYPredGenerated) fake = dlarray(zeros(1,size(dlYPred,2)));
real = dlarray(ones(1,size(dlYPred,2)));

D_loss = mean(sigmoid_cross_entropy_with_logits(dlYPredGenerated,fake)); D_loss = D_loss + mean(sigmoid_cross_entropy_with_logits(dlYPred,real)); lossDiscriminator = D_loss / 2;
end

function lossGenerator = ganGeneratorLoss(dlYPredGenerated) real = dlarray(ones(1,size(dlYPredGenerated,2)));
lossGenerator = mean(sigmoid_cross_entropy_with_logits(dlYPredGenerated,real));
end
function discriminatorParameters = initializeDiscriminatorWeights filterSize = [5 5];
dim = 64;

% Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 1 dim]); bias = zeros(1,1,dim,'single'); discriminatorParameters.Conv1.Weights = dlarray(weights); discriminatorParameters.Conv1.Bias = dlarray(bias);

% Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) dim 2*dim]); bias = zeros(1,1,2*dim,'single'); discriminatorParameters.Conv2.Weights = dlarray(weights); discriminatorParameters.Conv2.Bias = dlarray(bias);

% Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 2*dim 4*dim]); bias = zeros(1,1,4*dim,'single'); discriminatorParameters.Conv3.Weights = dlarray(weights); discriminatorParameters.Conv3.Bias = dlarray(bias);

% Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 4*dim 8*dim]); bias = zeros(1,1,8*dim,'single'); discriminatorParameters.Conv4.Weights = dlarray(weights); discriminatorParameters.Conv4.Bias = dlarray(bias);

% Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 8*dim 16*dim]); bias = zeros(1,1,16*dim,'single'); discriminatorParameters.Conv5.Weights = dlarray(weights); discriminatorParameters.Conv5.Bias = dlarray(bias);

% fully connected
weights = iGlorotInitialize([1,4 * 4 * dim * 16]); bias = zeros(1,1,'single'); discriminatorParameters.FC.Weights = dlarray(weights); discriminatorParameters.FC.Bias = dlarray(bias);
end

function generatorParameters = initializeGeneratorWeights dim = 64;
% Dense 1
weights = iGlorotInitialize([dim*256,100]); bias = zeros(dim*256,1,'single');
generatorParameters.FC.Weights = dlarray(weights); generatorParameters.FC.Bias = dlarray(bias);

filterSize = [5 5];

% Trans Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 8*dim 16*dim]); bias = zeros(1,1,dim*8,'single');
generatorParameters.Conv1.Weights = dlarray(weights); generatorParameters.Conv1.Bias = dlarray(bias);

% Trans Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 4*dim 8*dim]); bias = zeros(1,1,dim*4,'single');
generatorParameters.Conv2.Weights = dlarray(weights); generatorParameters.Conv2.Bias = dlarray(bias);

% Trans Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 2*dim 4*dim]); bias = zeros(1,1,dim*2,'single');
generatorParameters.Conv3.Weights = dlarray(weights); generatorParameters.Conv3.Bias = dlarray(bias);

% Trans Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) dim 2*dim]); bias = zeros(1,1,dim,'single');
generatorParameters.Conv4.Weights = dlarray(weights); generatorParameters.Conv4.Bias = dlarray(bias);

% Trans Conv2D
weights = iGlorotInitialize([filterSize(1) filterSize(2) 1 dim]); bias = zeros(1,1,1,'single');
generatorParameters.Conv5.Weights = dlarray(weights);
generatorParameters.Conv5.Bias = dlarray(bias); end

function out = sigmoid_cross_entropy_with_logits(x,z) out = max(x, 0) - x .* z + log(1 + exp(-abs(x))); end

function w = iGlorotInitialize(sz) if numel(sz) == 2
numInputs = sz(2); numOutputs = sz(1);
else
numInputs = prod(sz(1:3)); numOutputs = prod(sz([1 2 4]));
end
multiplier = sqrt(2 / (numInputs + numOutputs));
w = multiplier * sqrt(3) * (2 * rand(sz,'single') - 1); end







keyboardFolder = 'C:\Users\Shreepad Sapate\AppData\Local\Temp\TheKeyboard'; ads = audioDatastore(keyboardFolder,'IncludeSubfolders',true);

fftLength = 256;
win = hann(fftLength,'periodic'); overlapLength = 128;
if ~isempty(ver('parallel')) pool = gcp;
numPar = numpartitions(ads,pool); else
numPar = 1;
end
parfor ii = 1:numPar

subds = partition(ads,numPar,ii);
STrain = zeros(fftLength/2+1,128,1,numel(subds.Files)); for idx = 1:numel(subds.Files)
x = read(subds);

if length(x) > fftLength*64
% Lengthen the signal if it is too short x = x(1:fftLength*64);
end

% Convert from double-precision to single-precision x = single(x);

% Scale the signal
x = x ./ max(abs(x));

% Zero-pad to ensure stft returns 128 windows. x = [x ; zeros(overlapLength,1,'like',x)];

S0 =
stft(x,'Window',win,'OverlapLength',overlapLength,'Centered',false);

% Convert from two-sided to one-sided. S = S0(1:129,:);
S = abs(S);

STrain(:,:,:,idx) = S;
end
STrainC{ii} = STrain;
end
STrain = cat(4,STrainC{:}); STrain = log(STrain + 1e-6);
SMean = mean(STrain,[2 3 4]);
SStd = std(STrain,1,[2 3 4]);
STrain = (STrain-SMean)./SStd; STrain = STrain/3;
Y = reshape(STrain,numel(STrain),1); Y(Y<-1) = -1;
Y(Y>1) = 1;
STrain = reshape(Y,size(STrain)); STrain = STrain(1:end-1,:,:,:); STrain = permute(STrain,[2 1 3 4]);
maxEpochs = 1000;
miniBatchSize = 64;
numIterationsPerEpoch = floor(size(STrain,4)/miniBatchSize); learnRateGenerator = 0.0002;
learnRateDiscriminator = 0.0002;
gradientDecayFactor = 0.5;
squaredGradientDecayFactor = 0.999;
executionEnvironment = "auto"; generatorParameters = initializeGeneratorWeights;
discriminatorParameters = initializeDiscriminatorWeights;
trailingAvgGenerator = []; trailingAvgSqGenerator = []; trailingAvgDiscriminator = []; trailingAvgSqDiscriminator = [];
saveCheckpoints = false; numLatentInputs = 100;
iteration = 0;


%Training
for epoch = 1:maxEpochs

% Shuffle the data.
idx = randperm(size(STrain,4)); STrain = STrain(:,:,:,idx);

% Loop over mini-batches.
for index = 1:numIterationsPerEpoch iteration = iteration + 1;

% Read mini-batch of data.
dlX = STrain(:,:,:,(index-1)*miniBatchSize+1:index*miniBatchSize); dlX = dlarray(dlX,'SSCB');

% Generate latent inputs for the generator network.
Z = 2 * ( rand(1,1,numLatentInputs,miniBatchSize,'single') - 0.5 ) ; dlZ = dlarray(Z);

% If training on a GPU, then convert data to gpuArray. if (executionEnvironment == "auto" && canUseGPU) ||
executionEnvironment == "gpu"
dlZ = gpuArray(dlZ); dlX = gpuArray(dlX);
end

% Evaluate the discriminator gradients using dlfeval and the
% |modelDiscriminatorGradients| helper function. gradientsDiscriminator = ...

dlfeval(@modelDiscriminatorGradients,discriminatorParameters,generatorParameter s,dlX,dlZ);
% Update the discriminator network parameters. [discriminatorParameters,trailingAvgDiscriminator,trailingAvgSqDiscriminator] =
...
adamupdate(discriminatorParameters,gradientsDiscriminator, ... trailingAvgDiscriminator,trailingAvgSqDiscriminator,iteration, ...

learnRateDiscriminator,gradientDecayFactor,squaredGradientDecayFactor);

% Generate latent inputs for the generator network.
Z = 2 * ( rand(1,1,numLatentInputs,miniBatchSize,'single') - 0.5 ) ; dlZ = dlarray(Z);

% If training on a GPU, then convert data to gpuArray. if (executionEnvironment == "auto" && canUseGPU) ||
executionEnvironment == "gpu"
dlZ = gpuArray(dlZ);
end

% Evaluate the generator gradients using dlfeval and the
% |modelGeneratorGradients| helper function. gradientsGenerator = ...

dlfeval(@modelGeneratorGradients,discriminatorParameters,generatorParameters,dl Z);







end

% Update the generator network parameters. [generatorParameters,trailingAvgGenerator,trailingAvgSqGenerator] = ...
adamupdate(generatorParameters,gradientsGenerator, ...
trailingAvgGenerator,trailingAvgSqGenerator,iteration, ... learnRateGenerator,gradientDecayFactor,squaredGradientDecayFactor);


% Every 10 iterations, save a training snapshot to a MAT file. if saveCheckpoints && mod(epoch,10)==0
fprintf('Epoch %d out of %d complete\n',epoch,maxEpochs);
% Save checkpoint in case training is interrupted. save('audiogancheckpoint.mat',...
'generatorParameters','discriminatorParameters',... 'trailingAvgDiscriminator','trailingAvgSqDiscriminator',... 'trailingAvgGenerator','trailingAvgSqGenerator','iteration');
end
end




numLatentInputs = 100;
dlZ = dlarray(2 * ( rand(1,1,numLatentInputs,1,'single') - 0.5 )); dlXGenerated = modelGenerator(dlZ,generatorParameters);
S = dlXGenerated.extractdata;

S = S.';
S = [S ; zeros(1,128)];
S = S * 3;
S = (S.*SStd) + SMean; S = exp(S);
S = [S; S(end-1:-1:2,:)];
S = [zeros(256,100) S zeros(256,100)];

myAudio = stftmag2sig(S,256, ... 'FrequencyRange','twosided', ... 'Window',hann(256,'periodic'), ... 'OverlapLength',128, ...
'MaxIterations',20, ...
'Method','fgla');
myAudio = myAudio./max(abs(myAudio),[],'all'); myAudio = myAudio(128*100:end-128*100);

sound(myAudio,fs)

t = (0:length(myAudio)-1)/fs; plot(t,myAudio)
grid on
xlabel('Time (s)') title('Synthesized GAN Sound')

figure stft(myAudio,fs,'Window',hann(256,'periodic'),'OverlapLength',128);
